# PTM Workshop

In [ ]:
from openff.toolkit import ForceField, Molecule, Topology

molecule = Molecule.from_smiles("CC(=O)Nc1ccc(cc1)O")
molecule.generate_conformers(n_conformers=1)
topology = Topology.from_molecules([molecule])

force_field = ForceField("openff-2.3.0.offxml")
interchange = force_field.create_interchange(topology)
interchange.minimize()

openmm_system = interchange.to_openmm_system()
openmm_topology = interchange.to_openmm_topology()
openmm_positions = interchange.positions.to_openmm()

In [ ]:
import openmm

temperature = 300.0 * openmm.unit.kelvin
friction_coefficient = 1.0 / openmm.unit.picosecond
step_size = 2.0 * openmm.unit.femtosecond

simulation = openmm.app.Simulation(
    openmm_topology,
    openmm_system,
    openmm.LangevinIntegrator(temperature, friction_coefficient, step_size),
)
simulation.context.setPositions(openmm_positions)
simulation.context.setVelocitiesToTemperature(simulation.integrator.getTemperature())

simulation.reporters.append(
    openmm.app.DCDReporter(file="trajectory.dcd", reportInterval=100)
)
simulation.step(10000)

In [ ]:
import mdtraj
import nglview

trajectory: mdtraj.Trajectory = mdtraj.load(
    "trajectory.dcd", top=mdtraj.Topology.from_openmm(interchange.to_openmm_topology())
)

view = nglview.show_mdtraj(trajectory)
view

## What just happened

```python
from openff.toolkit import ForceField, Molecule, Topology

molecule = Molecule.from_smiles("CC(=O)Nc1ccc(cc1)O")
molecule.generate_conformers(n_conformers=1)

topology = Topology.from_molecules([molecule])

force_field = ForceField("openff-2.3.0.offxml")
interchange = force_field.create_interchange(topology)
interchange.minimize()

openmm_system = interchange.to_openmm_system()
openmm_topology = interchange.to_openmm_topology()
openmm_positions = interchange.positions.to_openmm()
```

1. **Easy**: Minimized OpenMM system ready in 10 lines and 5 seconds starting from nothing but a SMILES string and some FOSS
2. **Explicit**: All force field parameters are explicit in the force field file, including PME, cutoffs, functional form, constraints...
3. **Chemical**: System construction and parametrization is based on chemistry, not particles and atom types, and chemical identities survive parametrization
4. **Universal**: System construction and parametrization is the same across all supported simulation engines
5. **Extensible**: All steps are modular and can be augmented with arbitrary Python code

Like what?

- Loading molecule objects and coordinates from cheminformatics toolkits like RDKit and OpenEye
- Loading prepared chemical systems from external packages for OpenFF parametrization
- Generating condensed phase systems from scratch with PackMol and/or OpenFold
- Combining Interchanges prepared from compatible force fields from other packages (eg. OpenMM's implementations of Amber lipids or nucleic acids)
- User-fit force fields (eg. with BespokeFit)

Upcoming: OpenFF 3 Rosemary

- Self consistent protein and drug-like small molecule force field
- Fast, scalable, conformation independent, Amber-compatible graph neural network charge model
- Now in early Alpha


In [ ]:
from datetime import datetime
from tempfile import NamedTemporaryFile

import mdtraj
import nglview
import openmm
from openff.interchange import Interchange
from openff.toolkit import ForceField, Molecule, Quantity, Topology

HYDROGEN_MASS = Quantity(1.007947, "amu")


def simulate_and_visualize(
    interchange: Interchange,
    steps: int = 10_000,
    temperature: Quantity = Quantity(300.0, "K"),
    friction_coefficient: Quantity = Quantity(1.0, "1/ps"),
    step_size: Quantity = Quantity(2.0, "fs"),
    hydrogen_mass: Quantity = HYDROGEN_MASS,
    pressure: None | Quantity = None,
    membrane_surface_tension: None | Quantity = None,
    report_interval: int = 100,
):
    print(f"{datetime.now()} Exporting...")
    additional_forces = []
    if pressure is not None:
        if membrane_surface_tension is None:
            barostat = openmm.MonteCarloBarostat(
                pressure.to_openmm(),
                temperature.to_openmm(),
            )
        else:
            barostat = openmm.MonteCarloMembraneBarostat(
                pressure.to_openmm(),
                membrane_surface_tension.to_openmm(),
                temperature.to_openmm(),
                openmm.MonteCarloMembraneBarostat.XYIsotropic,
                openmm.MonteCarloMembraneBarostat.ZFree,
            )
        additional_forces.append(barostat)

    simulation = interchange.to_openmm_simulation(
        integrator=openmm.LangevinIntegrator(
            temperature.to_openmm(),
            friction_coefficient.to_openmm(),
            step_size.to_openmm(),
        ),
        hydrogen_mass=hydrogen_mass.m_as("amu"),
        additional_forces=additional_forces,
    )
    simulation.context.setVelocitiesToTemperature(
        simulation.integrator.getTemperature()
    )
    print(f"{datetime.now()} Minimizing...")
    simulation.minimizeEnergy()
    with NamedTemporaryFile(suffix=".dcd") as f:
        simulation.reporters.append(
            openmm.app.DCDReporter(file=f.name, reportInterval=report_interval)
        )
        print(f"{datetime.now()} Simulating...")
        simulation.step(steps)
        print(f"{datetime.now()} Visualizing.")
        trajectory: mdtraj.Trajectory = mdtraj.load(
            f.name, top=mdtraj.Topology.from_openmm(interchange.to_openmm_topology())
        )
    return nglview.show_mdtraj(trajectory)

## Preparing a topology with PackMol

In [ ]:
from openff.interchange.components._packmol import UNIT_CUBE, pack_box
from openff.toolkit import ForceField, Molecule, Quantity

dlpc = Molecule.from_smiles(
    "CCCCCCCCCCCC(=O)OC[C@H](CO[P@](=O)([O-])OCC[N+](C)(C)C)OC(=O)CCCCCCCCCCC"
)
water = Molecule.from_smiles("O")
for atom in water.atoms:
    atom.metadata["residue_name"] = "HOH"

topology = pack_box(
    [dlpc, water],
    [25, 4000],
    target_density=Quantity(1.0, "g/mL"),
    box_shape=UNIT_CUBE,
    tolerance=Quantity(0.05, "nm"),
)
topology.visualize()

In [ ]:
sage_ff = ForceField("openff-2.3.0.offxml")
interchange = sage_ff.create_interchange(topology)

In [ ]:
w = simulate_and_visualize(
    interchange,
    steps=250_000,
    report_interval=10_000,
    hydrogen_mass=4 * HYDROGEN_MASS,  # HMR for larger step size
    step_size=Quantity(4.0, "fs"),
)
w.clear_representations()
w.add_representation("licorice", selection="not water")
w.add_representation("unitcell")
w

## Use Amber Lipids for the micelle

In [ ]:
from openff.toolkit import Topology

lipid_molecules, other_molecules = [], []
for mol in topology.molecules:
    if mol == dlpc:
        lipid_molecules.append(mol)
    else:
        other_molecules.append(mol)
lipid_top = Topology.from_molecules(lipid_molecules)
other_top = Topology.from_molecules(other_molecules)

lipid_top.box_vectors = topology.box_vectors
other_top.box_vectors = topology.box_vectors

other_interchange = sage_ff.create_interchange(other_top)

In [ ]:
import openmm

lipid_ff = openmm.app.ForceField("amber14/lipid17.xml")
system = lipid_ff.createSystem(
    lipid_top.to_openmm(),
    nonbondedMethod=openmm.app.PME,
    nonbondedCutoff=9 * openmm.unit.angstrom,
    switchDistance=8 * openmm.unit.angstrom,
    constraints=openmm.app.HBonds,
)

In [ ]:
from openff.interchange import Interchange

lipid_interchange = Interchange.from_openmm(system, lipid_top)
lipid_interchange.visualize()

In [ ]:
combined_interchange = other_interchange.combine(
    lipid_interchange
)  # TODO: Investigate why doing this in the other order breaks
combined_interchange.visualize()

In [ ]:
w = simulate_and_visualize(
    combined_interchange,
    steps=250_000,
    report_interval=10_000,
    hydrogen_mass=4 * HYDROGEN_MASS,  # HMR for larger step size
    step_size=Quantity(4.0, "fs"),
)
w.clear_representations()
w.add_representation("licorice", selection="not water")
w.add_representation("unitcell")
w

## Preparing a system with RDKit

## Loading a pre-prepared, post-translationally modified protein system from PDB

In [ ]:
from openff.pablo import STD_CCD_CACHE, topology_from_pdb
from openff.pablo.residue import BondDefinition

STD_CCD_CACHE.auto_download = True

topology = topology_from_pdb(
    "8gjs_prepared.pdb",
    residue_library=STD_CCD_CACHE.with_crosslink(
        residues=["0EH", "MK8"],
        linking_atoms=["CAT", "CE"],
        leaving_atoms=[["H22"], ["HEB"]],
        bond_order=1,
    ),
)

In [ ]:
topology.molecule(1).visualize("rdkit", show_all_hydrogens=False)

In [ ]:
topology.box_vectors

In [ ]:
topology.visualize()

In [ ]:
force_field = ForceField("openff_no_water-3.0.0-alpha0.offxml", "opc3.offxml")
interchange = force_field.create_interchange(topology)

In [ ]:
simulate_and_visualize(interchange)

## Semaglutide bound to receptor in a membrane with Amber lipids

In [ ]:
from openff.pablo import STD_CCD_CACHE

STD_CCD_CACHE._definitions["KUT"] = (STD_CCD_CACHE["KUT"][0],)
STD_CCD_CACHE._definitions["LYS"] = tuple(
    resdef for resdef in STD_CCD_CACHE["LYS"] if "-HXT" not in resdef.description
)

In [ ]:
STD_CCD_CACHE["KUT"][0].visualize()

In [ ]:
STD_CCD_CACHE["LYS"][0].visualize()

In [ ]:
from openff.pablo import STD_CCD_CACHE, ResidueDefinition, topology_from_pdb

STD_CCD_CACHE.auto_download = True

topology = topology_from_pdb(
    "7KI0_prepared_popc_kut_repositioned.pdb",
    # "7KI0_minimized_kut_repositioned_minimized_bilayer.pdb",
    residue_library=STD_CCD_CACHE.with_crosslink(
        residues=["LYS", "KUT"],
        linking_atoms=["NZ", "C33"],
        leaving_atoms=[["HZ2"], ["H61"]],
        bond_order=1,
    ),
    additional_definitions=[
        ResidueDefinition.anon_from_smiles(
            r"CCCCCCCCCCCCCCCC(=O)OC[C@H](COP(=O)([O-])OCC[N+](C)(C)C)OC(=O)CCCCCCC/C=C\CCCCCCCC"
        )
    ],
)

In [ ]:
topology.molecule(0).visualize("nglview")

In [ ]:
w = topology.visualize()
w.clear_representations()
w.add_cartoon()
w.add_representation("licorice", selection="not water")
w.add_representation("unitcell")
w.add_spacefill(selection="water or ion", opacity=0.05)
w

In [ ]:
from openff.toolkit import Molecule, Topology

popc = Molecule.from_smiles(
    r"CCCCCCCCCCCCCCCC(=O)OC[C@H](COP(=O)([O-])OCC[N+](C)(C)C)OC(=O)CCCCCCC/C=C\CCCCCCCC",
    allow_undefined_stereo=True,
)
lipid_molecules = []
other_molecules = []
for mol in topology.molecules:
    if mol.hill_formula == popc.hill_formula:
        lipid_molecules.append(mol)
    else:
        other_molecules.append(mol)
lipid_top = Topology.from_molecules(lipid_molecules)
other_top = Topology.from_molecules(other_molecules)

lipid_top.box_vectors = topology.box_vectors
other_top.box_vectors = topology.box_vectors

In [ ]:
from openff.toolkit import ForceField, Molecule, Topology

rosemary_alpha_opc_ff = ForceField("openff_no_water-3.0.0-alpha0.offxml", "opc3.offxml")
other_interchange = rosemary_alpha_opc_ff.create_interchange(other_top)

In [ ]:
import openmm
from openff.interchange import Interchange

lipid_ff = openmm.app.ForceField("amber14/lipid17.xml")
system = lipid_ff.createSystem(
    lipid_top.to_openmm(),
    nonbondedMethod=openmm.app.PME,
    nonbondedCutoff=9 * openmm.unit.angstrom,
    switchDistance=8 * openmm.unit.angstrom,
    constraints=openmm.app.HBonds,
)
lipid_interchange = Interchange.from_openmm(system, lipid_top)
combined_interchange = other_interchange.combine(lipid_interchange)

In [ ]:
w = simulate_and_visualize(
    combined_interchange,
    steps=10_000,
    report_interval=100,
    hydrogen_mass=1 * HYDROGEN_MASS,  # HMR for larger step size
    step_size=Quantity(2, "fs"),
    pressure=Quantity(1, "bar"),
    membrane_surface_tension=Quantity(200, "bar nm"),
)
w.clear_representations()
w.add_cartoon()
w.add_representation("licorice", selection="not water")
# w.add_representation("unitcell")
w.add_spacefill(selection="water or ion", opacity=0.05)
w

In [ ]:
w

In [ ]:
steps = 1_000_000
report_interval = 1
hydrogen_mass = 1 * HYDROGEN_MASS
step_size = Quantity(1, "fs")
pressure = Quantity(1, "bar")
membrane_surface_tension = Quantity(200, "bar nm")
temperature = Quantity(300, "kelvin")
friction_coefficient = Quantity(1.0, "1/ps")

additional_forces = [
    openmm.MonteCarloMembraneBarostat(
        pressure.to_openmm(),
        membrane_surface_tension.to_openmm(),
        temperature.to_openmm(),
        openmm.MonteCarloMembraneBarostat.XYIsotropic,
        openmm.MonteCarloMembraneBarostat.ZFree,
    )
]

simulation = combined_interchange.to_openmm_simulation(
    integrator=openmm.LangevinIntegrator(
        temperature.to_openmm(),
        friction_coefficient.to_openmm(),
        step_size.to_openmm(),
    ),
    hydrogen_mass=hydrogen_mass.m_as("amu"),
    additional_forces=additional_forces,
)
simulation.context.setVelocitiesToTemperature(simulation.integrator.getTemperature())

In [ ]:
simulation.minimizeEnergy()

In [ ]:
simulation.reporters.append(
    openmm.app.DCDReporter(file="trajectory.dcd", reportInterval=report_interval)
)

simulation.step(steps)


In [ ]:
trajectory: mdtraj.Trajectory = mdtraj.load(
    "trajectory.dcd", top=mdtraj.Topology.from_openmm(interchange.to_openmm_topology())
)

In [ ]:
simulation.context.getState(positions=True).getPositions()